In [286]:
import pandas as pd
import numpy as np
import scipy.sparse.csr as csr
import scipy.sparse as sparse
import re
import matplotlib.pyplot as plt
import timeit
import os
import difflib 
import quandl
from fuzzywuzzy import fuzz, process
import ngram
import pickle
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import linear_model

quandl.ApiConfig.api_key = 'yd_TjLR5yNUyn_5TuXdP' 

### Load Electronics product metadata from pickle

In [330]:
dfElect = pd.read_pickle(os.getcwd() + r'\pickle\dfElect.pkl')

In [331]:
dfElect['brand'] = dfElect['brand'].str.lower()

In [105]:
def cleanCompany(name):
    name = name.split()[0].strip().replace(',','')
    if len(name) > 3:
        return name.lower()
    else:
        return ''

In [122]:
stocklist = pd.read_csv(os.getcwd() + r'\yahoo_stocklist.csv',header=0, encoding='latin-1', 
                        usecols=["YAHOO TICKER","COMPANY NAME", 'COUNTRY TRADED'])
stocklist = stocklist[stocklist['COUNTRY TRADED'] == 'United States of America']
stocklist = stocklist[~stocklist['COMPANY NAME'].isnull()]
stocklist['Company'] = stocklist['COMPANY NAME'].apply(cleanCompany)
stocklist = stocklist[stocklist['Company'] != '']
stocklist.rename(columns={'YAHOO TICKER': 'Ticker'}, inplace=True)
stocklist.head(10)

,Ticker,COMPANY NAME,COUNTRY TRADED,Company
4,AGU,Agrium Inc.,United States of America,agrium
6,AVD,American Vanguard Corp.,United States of America,american
18,CVAT,"Cavitation Technologies, Inc.",United States of America,cavitation
20,CERE,"Ceres, Inc.",United States of America,ceres
31,CGA,"China Green Agriculture, Inc.",United States of America,china
58,GROG,"GroGenesis, Inc.",United States of America,grogenesis
71,IPI,"Intrepid Potash, Inc.",United States of America,intrepid
78,KWBT,Kiwa Bio-Tech Products Group C,United States of America,kiwa
85,MBII,"Marrone Bio Innovations, Inc.",United States of America,marrone
96,MON,Monsanto Company,United States of America,monsanto


In [332]:
brands = dfElect.groupby('brand').size().reset_index()
brands.head(10)

,brand,0
0,1800battery,19
1,180s,7
2,2cool,4
3,2gig,14
4,2k,11
5,2wire,3
6,2xl,11
7,3 legged thing,7
8,3cleader,47
9,3com,9


In [109]:
def findMatch(company):
    return process.extractOne(company, stocklist['Company'].values.tolist(), scorer=fuzz.ratio, score_cutoff=95)

In [68]:
if 0:
    G = ngram.NGram(stocklist['Company'].values.tolist())
    brands['Company'] = brands['brand'].iloc[:100].apply(lambda x: G.find(x))

In [113]:
start_time = timeit.default_timer()

brands['Company'] = brands['brand'].apply(findMatch)

print('{0:.2f} seconds'.format(timeit.default_timer() - start_time))

266.75 seconds


In [166]:
brands.to_pickle('matched_brands.pkl')

In [334]:
brands = pd.read_pickle('matched_brands.pkl')

In [336]:
brands = brands[~brands['Company'].isnull()]

In [119]:
brands['Company'] = brands['Company'].apply(lambda x: x[0])

In [337]:
brands.head()

,brand,0,Company,Ticker,2009,2010,2011,2012,2013,2014
0,sony,2164,sony,SNE,NaN,NaN,NaN,NaN,NaN,NaN
1,canon,773,canon,CAJ,NaN,NaN,NaN,NaN,NaN,NaN
2,sandisk,664,sandisk,SNDK,2.019792,0.719903,-0.013037,-0.116033,0.633154,0.406362
3,garmin,598,garmin,GRMN,0.640193,0.051083,0.346101,0.067355,0.187106,0.183989
4,intel,525,intel,INTC,0.439035,0.062574,0.193316,-0.120336,0.308792,0.443015


In [124]:
brands = brands.merge(stocklist.iloc[:,[0,3]], on='Company')

In [162]:
def get_return(ticker):
    try:
        prices = quandl.get("WIKI/" + ticker, start_date="2008-1-1", end_date="2015-1-1", collapse='annual')['Adj. Close']
        returns = ((prices/prices.shift(1)) - 1).iloc[1:].values.tolist()
        return pd.Series(returns, index=[2009,2010,2011, 2012, 2013, 2014])
    except Exception:
        return pd.Series([np.nan]*6, index=[2009,2010,2011, 2012, 2013, 2014])

In [164]:
start_time = timeit.default_timer()
brands[[2009,2010,2011, 2012, 2013, 2014]] = brands['Ticker'].apply(get_return)
print('{0:.2f} seconds'.format(timeit.default_timer() - start_time))

190.86 seconds


In [340]:
brands.head(10)

,brand,0,Company,Ticker,2009,2010,2011,2012,2013,2014
2,sandisk,664,sandisk,SNDK,2.019792,0.719903,-0.013037,-0.116033,0.633154,0.406362
3,garmin,598,garmin,GRMN,0.640193,0.051083,0.346101,0.067355,0.187106,0.183989
4,intel,525,intel,INTC,0.439035,0.062574,0.193316,-0.120336,0.308792,0.443015
5,apple,434,apple,AAPL,1.469033,0.530665,0.255580,0.325884,0.080603,0.406174
6,monster,423,monster,MNST,0.145243,0.361458,0.762433,0.146950,0.282551,0.598790
9,plantronics,360,plantronics,PLT,0.991212,0.441967,-0.036960,0.045406,0.271431,0.155080
10,pioneer,351,pioneer,PXD,1.987007,0.804814,0.031640,0.192137,0.727812,-0.191001
14,seagate,347,seagate,STX,3.134876,-0.173722,0.130114,0.952939,0.898282,0.224367
15,netgear,264,netgear,NTGR,0.900964,0.552789,-0.003266,0.174561,-0.164595,0.080146
16,cisco,263,cisco,CSCO,0.468712,-0.154971,-0.096302,0.115957,0.167249,0.279088


In [338]:
brands = brands[~brands[2009].isnull()]

In [339]:
brands.drop_duplicates('brand', inplace=True)

#### Load G1, G2, G3 Electronics Matrices from pickle

In [175]:
def load_G_pickle(tag):
    path = os.getcwd() + r'\pickle\\'
    return pickle.load(open(path + tag + '_G1_Brand.pkl', "rb" )), \
           pickle.load(open(path + tag + '_G2_Brand.pkl', "rb" )), \
           pickle.load(open(path + tag + '_G3_Brand.pkl', "rb" ))

In [179]:
E_G1_Brand, E_G2_Brand, E_G3_Brand = load_G_pickle('E')

In [303]:
if 0:
    path = os.getcwd() + r'\pickle'

    E_bag_P = pickle.load(open(path + r'\P_Elec.pkl', "rb" ))
    E_bag_Q = pickle.load(open(path + r'\Q_Elec.pkl', "rb" ))
    E_bag_R = pickle.load(open(path + r'\R_Elec.pkl', "rb" ))

In [304]:
if 0:
    for year in range(1999,2009):
        E_bag_P.pop(year)
        E_bag_Q.pop(year)
        E_bag_R.pop(year)

In [298]:
if 0:
    filepath = r'C:\Users\David\Google Drive (deh284@nyu.edu)\datasets\amazon\processed'

    cols = ['productid','title','brand']
    dfElectOriginal = pd.read_csv(filepath + r'\Electronics\df_products_Elec.csv',usecols=cols)

    prod_Elect = dfElect.index.values
    row = []
    col = []
    k = 0
    for prod in range(dfElectOriginal.shape[0]): 
        if dfElectOriginal.iloc[prod,0] in prod_Elect:
            row.append(prod) 
            col.append(k)
            k += 1

    data = [1]*len(row)                           
    E_Reduce = sparse.csr_matrix((data,(row,col)),shape=(dfElectOriginal.shape[0],dfElect.shape[0])) 

In [305]:
if 0:
    for year in range(2009,2015):   
        E_bag_P[year] = E_bag_P[year] + E_bag_P[year].T
        E_bag_P[year] = E_Reduce.transpose().dot(E_bag_P[year]).dot(E_Reduce)
        E_bag_Q[year] = E_Reduce.transpose().dot(E_bag_Q[year]).dot(E_Reduce)
        E_bag_R[year] = E_Reduce.transpose().dot(E_bag_R[year]).dot(E_Reduce)

In [301]:
if 0:
    def prod2brandPQR(transform, bag_P, bag_Q, bag_R):
        bag_P_brand = {}
        bag_Q_brand = {}
        bag_R_brand = {}
        for y in bag_P.keys():
            bag_P_brand[y] = transform.transpose().dot(bag_P[y]).dot(transform)
            bag_Q_brand[y] = transform.transpose().dot(bag_Q[y]).dot(transform)
            bag_R_brand[y] = transform.transpose().dot(bag_R[y]).dot(transform)

        return bag_P_brand, bag_Q_brand, bag_R_brand

In [306]:
if 0:
    def prod2brand(df):
        row = []
        col = []
        brands = df['brand'].unique().tolist()   # Create list of all individual brands

        for prod in range(df.shape[0]):                # Iterate over all rows in product dataframe
            row.append(prod)                           # Add row for each individual product
            col.append(brands.index(df.iloc[prod,1]))  # Find related brand (column 1 -> see below) and append its index value

        data = [1]*len(row)                            # Define the amount of data (# of rows/products * 1 brand each)
        return sparse.csr_matrix((data,(row,col)),shape=(df.shape[0],len(brands))) # return n x m (products x brands) sparse matrix 

In [307]:
#E_Transform = prod2brand(dfElect)

In [310]:
#E_bag_P_bEquity, E_bag_Q_bEquity, E_bag_R_bEquity = prod2brandPQR(E_Transform, E_bag_P, E_bag_Q, E_bag_R)

#### Save Electronics Brand-level PQR to pickle 

In [313]:
if 0:
    path = os.getcwd() + r'\pickle'

    pickle.dump(E_bag_P_bEquity, open(path + r'\E_bag_P_bEquity.pkl', "wb" ))
    pickle.dump(E_bag_Q_bEquity, open(path + r'\E_bag_Q_bEquity.pkl', "wb" ))
    pickle.dump(E_bag_R_bEquity, open(path + r'\E_bag_R_bEquity.pkl', "wb" ))

#### Load Electronics Brand-level PQR from pickle

In [ ]:
path = os.getcwd() + r'\pickle'

E_bag_P_bEquity = pickle.load(open(path + r'\E_bag_P_bEquity.pkl', "rb" ))
E_bag_Q_bEquity = pickle.load(open(path + r'\E_bag_Q_bEquity.pkl', "rb" ))
E_bag_R_bEquity = pickle.load(open(path + r'\E_bag_R_bEquity.pkl', "rb" ))

#### Look for brands that have associated equity returns information 

In [341]:
validRows = [indx for indx, x in enumerate(dfElect['brand'].str.lower().unique().tolist()) \
             if x.lower() in brands['brand'].values.tolist()]

#### Align rows of graph matrices with appropriate equity values

In [346]:
brandList = dfElect['brand'].str.lower().unique().tolist()
stockList = brands['brand'].values.tolist()
brandIndx = [stockList.index(x) for x in brandList if x in stockList]

In [348]:
len(brandIndx)

138

In [314]:
E_bag_P_brand.keys()

dict_keys([2009, 2010, 2011, 2012, 2013, 2014])

### Build, Fit & Run predictive return models

#### Run Truncated SVD on Brand-level Electronics G1, G2, G3 & PQR, Define train (2009-2013) & test (2014) data 

In [351]:
n_components = 1

svd_E_G1 = TruncatedSVD(n_components, n_iter=5, random_state=42)
svd_E_G1.fit(E_G1_Brand)

svd_E_G2 = TruncatedSVD(n_components, n_iter=5, random_state=42)
svd_E_G2.fit(E_G2_Brand)

svd_E_G3 = TruncatedSVD(n_components, n_iter=5, random_state=42)
svd_E_G3.fit(E_G3_Brand)

X_train = np.zeros((0,6))
y_train = np.zeros((0))
X_test = np.zeros((0,6))
y_test = np.zeros((0))
for year in E_bag_P_brand.keys():
    
    svd_E_P = TruncatedSVD(n_components, n_iter=5, random_state=42)
    svd_E_P.fit(E_bag_P_brand[year])

    svd_E_Q = TruncatedSVD(n_components, n_iter=5, random_state=42)
    svd_E_Q.fit(E_bag_Q_brand[year])

    svd_E_R = TruncatedSVD(n_components, n_iter=5, random_state=42)
    svd_E_R.fit(E_bag_R_brand[year])
    
    X_Year = np.concatenate((svd_E_G1.components_[:,validRows].T, svd_E_G2.components_[:,validRows].T,
                             svd_E_G3.components_[:,validRows].T, svd_E_P.components_[:,validRows].T,
                             svd_E_Q.components_[:,validRows].T,  svd_E_R.components_[:,validRows].T), axis=1)
    if year == 2014:
        X_test = np.concatenate((X_test,X_Year),axis=0)
        y_test = np.concatenate((y_test,brands[year].iloc[brandIndx]),axis=0)
    else:
        X_train = np.concatenate((X_train,X_Year),axis=0)
        y_train = np.concatenate((y_train,brands[year].iloc[brandIndx]),axis=0)

In [317]:
X_train.shape

(690, 6)

In [323]:
y_train.shape

(690,)

In [357]:
np.corrcoef(y_train, X_train[:,5])

array([[ 1.        , -0.00332472],
       [-0.00332472,  1.        ]])

#### Fit Regression models

In [363]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123)
   
regr_lin = linear_model.LinearRegression()
print('R2 for OLS:', regr_lin.fit(X_train, y_train).score(X_test, y_test))

regr_fst = RandomForestRegressor()
print('R2 for Random Forest:', regr_fst.fit(X_train, y_train).score(X_test, y_test))

regr_bst = GradientBoostingRegressor()
print('R2 for Gradient Boosting:', regr_bst.fit(X_train, y_train).score(X_test, y_test))

R2 for OLS: -0.685769858652
R2 for Random Forest: -1.28979153987
R2 for Gradient Boosting: -0.666942314862
